### Import Libraries

In [1]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import AlexNet

### Test for CUDA
Make sure to install the correct CUDA version and packages, see: https://pytorch.org/get-started/locally/

In [3]:
# check if CUDA is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device}')

Using cuda


### Load Images

#### Create the transformer
We resize the image from 640x640 to 32x32, convert them to a tensor, and normalize them with the default values from the ImageNet dataset.

In [4]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for AlexNet
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

#### Load the dataset and dataloader

In [5]:
images_path = 'data/images'
dataset = datasets.ImageFolder(root=images_path, transform=data_transforms)

#### Show some images and labels.

In [6]:
print(dataset.classes)
print(len(dataset.classes))
print(len(dataset))

# for images, labels in dataloader:
#     print(images)
#     print(labels)

['Black bishop', 'Black king', 'Black knight', 'Black pawn', 'Black queen', 'Black rook', 'White bishop', 'White king', 'White knight', 'White pawn', 'White queen', 'White rook']
12
300


### Split Train and Test
We split test 0.2, to train 0.8.

In [7]:
ratio = 0.8
train_size = int(ratio * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

#### Create the dataloaders
We set a feasible batch size (Amount of images is 300).
It is important to create the dataloaders after the split!

In [8]:
print(len(train_dataset))
print(len(test_dataset))

240
60


In [9]:
# Train set is 240 images, test set is 60 images. Batch size to 20
batch_size = 20

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Do the training with AlexNet

### Init it

In [12]:
# Define the network: AlexNet, num_classes is the number of classes in the dataset (from the dataset directly)
net = AlexNet(num_classes=len(dataset.classes))
net.to(device)

# specify loss function
criterion = torch.nn.CrossEntropyLoss()

# Optimizer and momentum
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Number of epochs
num_epochs = 100

### DO THE TRAINING!!
